In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import random

from DataLoader import load_data
from Agent import Actor, sample_action

In [2]:
movies, ratings, user_embeddings, movie_embeddings, genre_embeddings_matrix, \
    movieId_to_genre_index, movieId_to_index, index_to_movieId, \
    unique_user_ids, userId_to_index, index_to_userId = load_data()

In [3]:
actor = Actor(input_dim=user_embeddings.shape[1] + genre_embeddings_matrix.shape[1],
                  output_dim=len(movie_embeddings))
actor.load_state_dict(torch.load("trained_actor2.pt"))
actor.eval()

Actor(
  (fc1): Linear(in_features=60, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (output): Linear(in_features=64, out_features=77877, bias=True)
)

In [7]:
raw_movies = pd.read_csv('/Users/isaaclevy/Desktop/MSDS/DS7540/project/scripts/ml-25m/movies.csv')
movieId_to_title = dict(zip(raw_movies['movieId'], raw_movies['title']))
movieId_to_genre = dict(zip(raw_movies['movieId'], raw_movies['genres']))
movies = pd.read_csv('/Users/isaaclevy/Desktop/MSDS/DS7540/project/scripts/ml-25m/movies.csv')
ratings = pd.read_csv('/Users/isaaclevy/Desktop/MSDS/DS7540/project/scripts/ml-25m/ratings.csv')

In [8]:
# --- Build state vector ---
def get_state(user_embed, history, genre_embeddings_matrix, movieId_to_genre_index, index_to_movieId):
    history_genre_embeds = []

    for movie_index in history:
        movie_id = index_to_movieId[movie_index]
        if movie_id in movieId_to_genre_index:
            genre_index = movieId_to_genre_index[movie_id]
            if 0 <= genre_index < genre_embeddings_matrix.shape[0]:
                history_genre_embeds.append(torch.tensor(genre_embeddings_matrix[genre_index], dtype=torch.float32))

    if history_genre_embeds:
        history_genre_embed = torch.stack(history_genre_embeds).mean(dim=0)
    else:
        history_genre_embed = torch.zeros(genre_embeddings_matrix.shape[1], dtype=torch.float32)

    return torch.cat([user_embed, history_genre_embed], dim=0)

In [59]:
random_user_id = random.choice(ratings['userId'].unique())
user_ratings = ratings[ratings['userId'] == random_user_id]
user_index = userId_to_index[random_user_id]
user_embed = torch.tensor(user_embeddings.mean(axis=0), dtype=torch.float32)

user_movieIds = user_ratings['movieId'].tolist()
user_movie_indices = [movieId_to_index[mid] for mid in user_movieIds if mid in movieId_to_index]

if not user_movie_indices:
    raise ValueError(f"No rated movies found in index for user {random_user_id}")

initial_movie_index = random.choice(user_movie_indices)
history = [initial_movie_index]

num_ratings_available = len(user_ratings)
num_recommendations = int(num_ratings_available / 2)
print(f'Number of ratings available for current user: {num_ratings_available}')
print(f'Loop length: {num_recommendations}')

Number of ratings available for current user: 155
Loop length: 77


In [60]:
user_ratings['rating'].describe()

count    155.000000
mean       3.632258
std        1.056946
min        1.000000
25%        3.000000
50%        4.000000
75%        4.000000
max        5.000000
Name: rating, dtype: float64

In [61]:
print(f"Testing on user {random_user_id} ({index_to_userId[user_index]})")
print(f"Initial movie: {movieId_to_title[index_to_movieId[initial_movie_index]]}")
print(f"Initial movie rating: {user_ratings[user_ratings['movieId'] == index_to_movieId[initial_movie_index]].iloc[0]['rating']}")

Testing on user 60411 (60411)
Initial movie: Wedding Singer, The (1998)
Initial movie rating: 4.0


In [62]:
rating_history = []
for step in range(num_recommendations):
    state = get_state(user_embed, history, genre_embeddings_matrix, movieId_to_genre_index, index_to_movieId)
    
    with torch.no_grad():
        probs = actor(state)
        probs = torch.softmax(probs, dim=0)
    
    available_movies = list(set(user_movie_indices) - set(history))
    if not available_movies:
        print("No more movies")
        break
    
    next_movie_index = sample_action(probs, available_movies)
    next_movie_id = index_to_movieId[next_movie_index]
    
    history.append(next_movie_index)
    
    user_rating_row = user_ratings[user_ratings['movieId'] == next_movie_id]
    if not user_rating_row.empty:
        rating = user_rating_row.iloc[0]['rating']
    else:
        rating = None
    
    rating_history.append(rating)
    
    print(f"Step {step + 1}: Recommended '{movieId_to_title[next_movie_id]}', User Rating: {rating}")

Step 1: Recommended 'Sneakers (1992)', User Rating: 5.0
Step 2: Recommended 'Fight Club (1999)', User Rating: 5.0
Step 3: Recommended 'U.S. Marshals (1998)', User Rating: 3.0
Step 4: Recommended 'Species II (1998)', User Rating: 2.0
Step 5: Recommended 'Kingpin (1996)', User Rating: 4.0
Step 6: Recommended 'Batman & Robin (1997)', User Rating: 1.0
Step 7: Recommended 'Waterworld (1995)', User Rating: 4.0
Step 8: Recommended 'Pi (1998)', User Rating: 5.0
Step 9: Recommended 'Cool Runnings (1993)', User Rating: 3.0
Step 10: Recommended 'Deuce Bigalow: Male Gigolo (1999)', User Rating: 4.0
Step 11: Recommended 'Silence of the Lambs, The (1991)', User Rating: 5.0
Step 12: Recommended 'Big Daddy (1999)', User Rating: 3.0
Step 13: Recommended 'Thin Red Line, The (1998)', User Rating: 4.0
Step 14: Recommended 'Cable Guy, The (1996)', User Rating: 5.0
Step 15: Recommended 'Independence Day (a.k.a. ID4) (1996)', User Rating: 4.0
Step 16: Recommended 'Pretty Woman (1990)', User Rating: 3.0
Step 

In [63]:
pd.DataFrame({'Ratings': rating_history}).describe()

,Ratings
count,77.000000
mean,3.545455
std,1.058120
min,1.000000
25%,3.000000
50%,4.000000
75%,4.000000
max,5.000000
